<a href="https://colab.research.google.com/github/morarez/dataset-recommender-system/blob/main/artifacts_Doc2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)
import pandas as pd
import numpy as np
from gensim import models

Mounted at /content/gdrive


In [ ]:
#df = pd.read_json("/content/gdrive/MyDrive/Recommendation_Systems_Activities/TPDL2022/Data/data_with_relations_v1/relations.json",lines=True)
#df = pd.read_json("/content/gdrive/MyDrive/Recommendation_Systems_Activities/TPDL2022/Data/data_with_relations_v1/cleaned_concat_result.json")
#df = pd.read_json('/content/gdrive/MyDrive/Recommendation_Systems_Activities/Journal/Data/Metadata_21june/metadata_v3.json')

df = pd.read_json('/content/gdrive/MyDrive/Recommendation_Systems_Activities/Journal/Data/ni_3900/metadata_ni_3900.json', orient='index')

In [ ]:
df.reset_index(drop=True, inplace=True)
#df

In [ ]:
df["text"] = df["en_abstract"] + " " +  df['en_title']  

## Pre-processing

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(lemma_words)


df['cleanText']=df['text'].map(lambda s:preprocess(s)) 

## Doc2Vec

In [ ]:
data = df['cleanText']

In [ ]:
pip install Gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def tagged_document(list_of_list_of_words):
   for i, list_of_words in enumerate(list_of_list_of_words):
      yield models.doc2vec.TaggedDocument(list_of_words, [i])
data_training = list(tagged_document(data))

In [ ]:
model = models.doc2vec.Doc2Vec(vector_size=128, min_count=1, epochs=100)

In [ ]:
model.build_vocab(data_training)

In [ ]:
model.train(data_training, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
#from gensim.test.utils import get_tmpfile

In [ ]:
#fname = get_tmpfile("/content/gdrive/MyDrive/Recommendation_Systems_Activities/Journal/Notebooks/Trained_models/doc2vec_model_23june")

In [ ]:
#model.save(fname)

In [ ]:
##MAIN FUNCTION##
def rec_user1(user_list):
  import numpy as np
  user_profile = []
  for user_id in user_list:
    user_index = df[df['id']==user_id].index
    user_profile.append(model.docvecs[user_index[0]])
  column_average = [sum(sub_list) / len(sub_list) for sub_list in zip(*user_profile)]
  user_vector = numpy.array(column_average)

  similars = model.docvecs.most_similar(positive=[user_vector],topn=100)

  rec_list = []
  for doc in similars:
    rec_index = doc[0]
    rec_id = df.iloc[rec_index]['id']
    if df.loc[df['id'] == rec_id, 'type'].iloc[0] == 'software' or df.loc[df['id'] == rec_id, 'type'].iloc[0] == 'dataset':
      rec_list.append(rec_id)

  main_list = [x for x in rec_list if x not in user_list]

  return main_list[:10]

In [ ]:
#def rec_user(user_list):
  # user_profile = []
  # for user_id in user_list:
  #   user_index = df[df['id']==user_id].index
  #   user_profile.append(model.docvecs[user_index[0]])
  # column_average = [sum(sub_list) / len(sub_list) for sub_list in zip(*user_profile)]
  # user_vector = numpy.array(column_average)

  # similars = model.docvecs.most_similar(positive=[user_vector],topn=100)

  # rec_list = []
  # for doc in similars:
  #   rec_index = doc[0]
  #   rec_id = df.iloc[rec_index]['id']
  #   if df.loc[df['id'] == rec_id, 'type'].iloc[0] == 'software' or df.loc[df['id'] == rec_id, 'type'].iloc[0] == 'dataset':
  #     rec_list.append((rec_id,doc[1],df.loc[df['id'] == rec_id, 'type'].iloc[0]))
  # return rec_list[:10]


In [ ]:
#def recommendation(my_id):
  # newid = df[df['id']==my_id].index
  # similars =  model.docvecs.most_similar(newid,topn=100)
  # rec_list = []
  # for doc in similars:
  #   rec_index = doc[0]
  #   rec_id = df.iloc[rec_index]['id']
  #   if df.loc[df['id'] == rec_id, 'type'].iloc[0] == 'software' or df.loc[df['id'] == rec_id, 'type'].iloc[0] == 'dataset':
  #     rec_list.append((rec_id,doc[1],df.loc[df['id'] == rec_id, 'type'].iloc[0]))
  # return rec_list[:10]

In [ ]:
#import numpy
# def user(user_list):
#   user_profile = []
#   for user_id in user_list:
#     user_index = df[df['id']==user_id].index
#     user_profile.append(model.docvecs[user_index[0]])
#   column_average = [sum(sub_list) / len(sub_list) for sub_list in zip(*user_profile)]
#   user_vector = numpy.array(column_average)

#   return user_vector

# Dataset RecSys

In [ ]:
def rec_user_d(user_list):
  import numpy as np
  user_profile = []
  for user_id in user_list:
    user_index = df[df['id']==user_id].index
    user_profile.append(model.docvecs[user_index[0]])
  column_average = [sum(sub_list) / len(sub_list) for sub_list in zip(*user_profile)]
  user_vector = np.array(column_average)

  similars = model.docvecs.most_similar(positive=[user_vector],topn=100)

  rec_list = []
  for doc in similars:
    rec_index = doc[0]
    rec_id = df.iloc[rec_index]['id']
    if df.loc[df['id'] == rec_id, 'type'].iloc[0] == 'dataset':
      rec_list.append(rec_id)

  main_list = [x for x in rec_list if x not in user_list]

  return main_list[:10]

##users

In [ ]:
#users = pd.read_json('/content/gdrive/MyDrive/Recommendation_Systems_Activities/Journal/Data/users_june21/users_v3.json')
#users = users.loc[users.astype(str).drop_duplicates(subset=['id']).index]
#users.reset_index(drop=True, inplace=True)
users_d = pd.read_json('/content/gdrive/MyDrive/Recommendation_Systems_Activities/Journal/Data/ni_3900/users_ni_3900.json',orient='index')

In [ ]:
def d_counter(user_prof):
  d_counter = 0
  for id in user_prof:
    item_type = df[df['id'] == id]['type']
    for i in item_type:
      if str(i) == 'dataset':
        d_counter += 1
  return d_counter

In [ ]:
users_d['d_counter'] = users_d['id'].apply(lambda row : d_counter(row))

In [ ]:
users_d = users_d[users_d['d_counter'] != 0 ]
users_d = users_d[users_d['d_counter'] != 1 ]
users_d = users_d[users_d['d_counter'] != 2 ]
users_d = users_d[users_d['d_counter'] != 3 ]

users_d.reset_index(drop=True, inplace=True)

In [ ]:
users_d

,author_name,id,d_counter
0,"A. Crone, Eveline",[50|r37980778c78::929a30378eb172e271714387c0d5...,4
1,"A. Jacobsson, Josefin",[50|r37980778c78::3733d784e60cb4a92ec8226b7564...,4
2,"A. Norman, Kenneth",[50|r37980778c78::3dde9a8ae6db8d35cdfb4fd5585c...,6
3,"A. Turner, Jessica",[50|r37980778c78::c0dbf0428951817d1855dd0b2c5a...,6
4,"Afzali, Maryam",[50|r37980778c78::6f1ef756cf61134a31340a05042f...,6
...,...,...,...
148,"Yamada, Takashi",[50|doi_dedup___::6f9d880df4bfd4c4aaa2c2fe0f39...,4
149,"Yang, Yang",[50|r37980778c78::6190c74f564a6118cad28ab5349e...,5
150,"Yu, Chunshui",[50|r37980778c78::3360403652895b1d322d59a72e11...,8
151,"Zhao, Zhilian",[50|r37980778c78::af00ac7d9359d7e4b018bf6887dc...,6


##Leave one out

In [ ]:
def leave_one_out_d(profile_items):
  for item in profile_items:
      item_type = df[df['id'] == item]['type'].tolist()
      if item_type[0] == 'dataset':
        profile_items.remove(item)
        return item

In [ ]:
users_d['leave_one_out_d'] = users_d['id'].apply(lambda x:leave_one_out_d(x))

##Leave two out

In [ ]:
## IMPORTANT: if leave-1-out/leave-3-out has already been run, you need to load users from the start

def leave_two_out_d(profile_items):
  dataset_list = []
  for item in profile_items:
      item_type = df[df['id'] == item]['type'].tolist()
      if item_type[0] == 'dataset':
          dataset_list.append(item)
          if len(dataset_list) == 2:
            for ds in dataset_list:
              profile_items.remove(ds)
            return dataset_list

In [ ]:
users_d['leave_two_out_d'] = users_d['id'].apply(lambda x:leave_two_out_d(x))

##Leave three out

In [ ]:
# ## IMPORTANT: if leave-1-out/leave-2-out has already been run, you need to load users from the start

def leave_three_out_d(profile_items):
  dataset_list = []
  for item in profile_items:
      item_type = df[df['id'] == item]['type'].tolist()
      if item_type[0] == 'dataset':
          dataset_list.append(item)
          if len(dataset_list)==3:
            for ds in dataset_list:
                profile_items.remove(ds)

            return dataset_list
  

In [ ]:
users_d['leave_three_out_d'] = users_d['id'].apply(lambda x:leave_three_out_d(x))

In [ ]:
users_d['id'][0]

['50|r37980778c78::218b9a7a6370f88875d05e1a65bafa11']

In [ ]:
users_d['leave_three_out_d'][0]

['50|r37980778c78::929a30378eb172e271714387c0d52d31',
 '50|r37980778c78::285b7a83deee0d0557c4d430c05ca786',
 '50|r37980778c78::96fc66a94507f7fa4cda989e11187b37']

##Dataset Recommender

In [ ]:
users_d['recommended_items'] = users_d['id'].apply(lambda s:rec_user_d(s)) 

## Hit-rate@1 Evaluation

In [ ]:
def hit_rate_d(row):
  for item in row["recommended_items"]:
    if item == row['leave_one_out_d']:
      return 1
  return 0 

In [ ]:
users_d['hit_rate_d'] = users_d.apply(lambda x:hit_rate_d(x),axis=1)

In [ ]:
users_d['hit_rate_d'].mean()

0.3202614379084967

## Hit-rate@2 Evaluation

In [ ]:
def hit_rate_d(row):
  for item in row["recommended_items"]:
    if item in row['leave_two_out_d']:
      return 1
  return 0 

In [ ]:
users_d['hit_rate_d'] = users_d.apply(lambda x:hit_rate_d(x),axis=1)
users_d['hit_rate_d'].mean()

0.3790849673202614

## Hit-rate@3 Evaluation

In [ ]:
def hit_rate_d(row):
  for item in row["recommended_items"]:
    if item in row['leave_three_out_d']:
      return 1
  return 0 

In [ ]:
users_d["leave_three_out_d"] 

0      [50|r37980778c78::929a30378eb172e271714387c0d5...
1      [50|r37980778c78::3733d784e60cb4a92ec8226b7564...
2      [50|r37980778c78::3dde9a8ae6db8d35cdfb4fd5585c...
3      [50|r37980778c78::c0dbf0428951817d1855dd0b2c5a...
4      [50|r37980778c78::6f1ef756cf61134a31340a05042f...
                             ...                        
148    [50|r37980778c78::13c50b8f03369ab5555af1a0b231...
149    [50|r37980778c78::6190c74f564a6118cad28ab5349e...
150    [50|r37980778c78::3360403652895b1d322d59a72e11...
151    [50|r37980778c78::af00ac7d9359d7e4b018bf6887dc...
152    [50|dedup_wf_001::3ddc4ab8e8415f5932dcc3e2273f...
Name: leave_three_out_d, Length: 153, dtype: object

In [ ]:
users_d['hit_rate_d'] = users_d.apply(lambda x:hit_rate_d(x),axis=1)
users_d['hit_rate_d'].mean()

0.45751633986928103

# Software RecSys

In [ ]:
def rec_user_sw(user_list):
  import numpy as np
  user_profile = []
  for user_id in user_list:
    user_index = df[df['id']==user_id].index
    user_profile.append(model.docvecs[user_index[0]])
  column_average = [sum(sub_list) / len(sub_list) for sub_list in zip(*user_profile)]
  user_vector = np.array(column_average)

  similars = model.docvecs.most_similar(positive=[user_vector],topn=100)

  rec_list = []
  for doc in similars:
    rec_index = doc[0]
    rec_id = df.iloc[rec_index]['id']
    if df.loc[df['id'] == rec_id, 'type'].iloc[0] == 'software':
      rec_list.append(rec_id)

  main_list = [x for x in rec_list if x not in user_list]

  return main_list[:10]

##users

In [ ]:
users_sw = pd.read_json('/content/gdrive/MyDrive/Recommendation_Systems_Activities/Journal/Data/ni_3900/users_ni_3900.json',orient='index')
users_sw = users_sw.loc[users_sw.astype(str).drop_duplicates(subset=['id']).index]
users_sw.reset_index(drop=True, inplace=True)

In [ ]:
def sw_counter(user_prof):
  sw_counter = 0
  for id in user_prof:
    item_type = df[df['id'] == id]['type']
    for i in item_type:
      if str(i) == 'software':
        sw_counter += 1
  return sw_counter

In [ ]:
users_sw['sw_counter'] = users_sw['id'].apply(lambda row : sw_counter(row))

In [ ]:
users_sw = users_sw[users_sw['sw_counter'] != 0 ]
users_sw = users_sw[users_sw['sw_counter'] != 1 ]
users_sw = users_sw[users_sw['sw_counter'] != 2 ]
users_sw = users_sw[users_sw['sw_counter'] != 3 ]
users_sw.reset_index(drop=True, inplace=True)

##Leave one out

In [ ]:
def leave_one_out_sw(profile_items):
   for item in profile_items:
      item_type = df[df['id'] == item]['type'].tolist()
      if item_type[0] == 'software':
        profile_items.remove(item)
        return item

In [ ]:
users_sw['leave_one_out_sw'] = users_sw['id'].apply(lambda x:leave_one_out_sw(x))

## Leave two out

In [ ]:
def leave_two_out_sw(profile_items):
  sw_list = []
  for item in profile_items:
      item_type = df[df['id'] == item]['type'].tolist()
      if item_type[0] == 'software':
          sw_list.append(item)
          if len(sw_list) == 2:
            for sw in sw_list:
              profile_items.remove(sw)
            return sw_list

In [ ]:
users_sw['leave_two_out_sw'] = users_sw['id'].apply(lambda x:leave_two_out_sw(x))

In [ ]:
users_sw['leave_two_out_sw'][0]

['50|doi_________::77d25c5587f0b1ab44b34485d4ecb37f',
 '50|doi_________::c1881ca8737b1dd96123c03e5f11e4c2']

In [ ]:
users_sw['id'][0]

['50|doi_________::db035d2dbe49ed7a1dc68f5163222c72',
 '50|doi_________::84c31f6e86fc012ff2f5725fc4553af6',
 '50|doi_________::b58ad48a3597070a30da7860cc939128',
 '50|doi_________::e23490325da3dce7da8d8d801ad5ee27',
 '50|doi_dedup___::d1e4df32d1acb2c2b81455857c8bb3fb',
 '50|doi_________::d0b3eddc114427a3e0e7f5effc818020',
 '50|doi_________::a24af867c17a79d1ff0af8c9896d7826',
 '50|doi_________::84b4201d70b77d4cf3afcb1313698b39',
 '50|doi_________::678fb5611994f25abfca1aaafce7a398',
 '50|doi_dedup___::65b5a171d46983aaf71588d8cb8c4c94',
 '50|doi_________::ba9436e30d9b921dd39ad41cf8d05993']

## Leave three out

In [ ]:
def leave_three_out_sw(profile_items):
  sw_list = []
  for item in profile_items:
      item_type = df[df['id'] == item]['type'].tolist()
      if item_type[0] == 'software':
          sw_list.append(item)
          if len(sw_list) == 3:
            for sw in sw_list:
              profile_items.remove(sw)
            return sw_list


In [ ]:
users_sw['leave_three_out_sw'] = users_sw['id'].apply(lambda x:leave_three_out_sw(x))

In [ ]:
users_sw['leave_three_out_sw'][0]

['50|doi_________::77d25c5587f0b1ab44b34485d4ecb37f',
 '50|doi_________::c1881ca8737b1dd96123c03e5f11e4c2',
 '50|doi_________::db035d2dbe49ed7a1dc68f5163222c72']

In [ ]:
users_sw['id'][0]

['50|doi_________::84c31f6e86fc012ff2f5725fc4553af6',
 '50|doi_________::b58ad48a3597070a30da7860cc939128',
 '50|doi_________::e23490325da3dce7da8d8d801ad5ee27',
 '50|doi_dedup___::d1e4df32d1acb2c2b81455857c8bb3fb',
 '50|doi_________::d0b3eddc114427a3e0e7f5effc818020',
 '50|doi_________::a24af867c17a79d1ff0af8c9896d7826',
 '50|doi_________::84b4201d70b77d4cf3afcb1313698b39',
 '50|doi_________::678fb5611994f25abfca1aaafce7a398',
 '50|doi_dedup___::65b5a171d46983aaf71588d8cb8c4c94',
 '50|doi_________::ba9436e30d9b921dd39ad41cf8d05993']

## SW Recommender

In [ ]:
users_sw['recommended_items'] = users_sw['id'].apply(lambda s:rec_user_sw(s)) 

## Hit-rate@1 

In [ ]:
def hit_rate_sw(row):
  for item in row["recommended_items"]:
    if item == row['leave_one_out_sw']:
      return 1
  return 0 


In [ ]:
users_sw['hit_rate'] = users_sw.apply(lambda x:hit_rate_sw(x),axis=1)

In [ ]:
users_sw['hit_rate'].mean()

0.7692307692307693

## Hit-rate@2

In [ ]:
def hit_rate_sw(row):
  for item in row["recommended_items"]:
    if item == row['leave_two_out_sw']:
      return 1
  return 0 


In [ ]:
users_sw['hit_rate'] = users_sw.apply(lambda x:hit_rate_sw(x),axis=1)

In [ ]:
users_sw['hit_rate'].mean()

0.0

In [ ]:
users_sw['hit_rate']

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
Name: hit_rate, dtype: int64

## Hit-rate@3

In [ ]:
def hit_rate_sw(row):
  for item in row["recommended_items"]:
    if item == row['leave_three_out_sw']:
      return 1
  return 0 


In [ ]:
users_sw['hit_rate'] = users_sw.apply(lambda x:hit_rate_sw(x),axis=1)

In [ ]:
users_sw['hit_rate'].mean()

0.0

## Users - OLD

In [ ]:
#users = pd.read_json("/content/gdrive/MyDrive/Recommendation_Systems_Activities/TPDL2022/Data/users.json",lines=True)
users = pd.read_json('/content/gdrive/MyDrive/Recommendation_Systems_Activities/Journal/Data/users_june21/users_v3.json')

In [ ]:
def clean_users(user_prof):
  pub_counter = 0
  sd_counter = 0
  for id in user_prof:
    item_type = df[df['id'] == id]['type']
    for i in item_type:
      if str(i) == 'publication':
        pub_counter += 1
      else:
        sd_counter += 1
  return sd_counter

In [ ]:
def clean_users1(user_prof):
  pub_counter = 0
  sd_counter = 0
  for id in user_prof:
    item_type = df[df['id'] == id]['type']
    for i in item_type:
      if str(i) == 'publication':
        pub_counter += 1
      else:
        sd_counter += 1
  return pub_counter

In [ ]:
users['sd_counter'] = users['id'].apply(lambda row : clean_users(row))

In [ ]:
users['pub_counter'] = users['id'].apply(lambda row : clean_users1(row))

In [ ]:
users = users[users['sd_counter'] != 0 ]

In [ ]:
users.reset_index(drop=True, inplace=True)

In [ ]:
users

,author_name,id,sd_counter,pub_counter
0,"A. Norman, Kenneth",[50|r37980778c78::3dde9a8ae6db8d35cdfb4fd5585c...,6,0
1,"A. Turner, Jessica",[50|r37980778c78::eca395a306186f95e5fbe3b16f68...,6,0
2,"Aja-Fernández, Santiago",[50|r37980778c78::b52f346497320489e3de542b6bdd...,9,1
3,"Amirbekian, Bago",[50|doi_________::c366b2de1b4e34c37362d5eb8f43...,14,0
4,"Amlien, Inge K.",[50|doi_dedup___::d1e4df32d1acb2c2b81455857c8b...,10,1
...,...,...,...,...
298,"Zhao, Zhilian",[50|r37980778c78::af00ac7d9359d7e4b018bf6887dc...,6,1
299,"Zhong, Ning",[50|r37980778c78::4fcd43dfa35532d0819423b5b7c4...,5,2
300,"Zhou, Haiyan",[50|r37980778c78::4fcd43dfa35532d0819423b5b7c4...,5,1
301,"de Luis-García, Rodrigo",[50|r37980778c78::b52f346497320489e3de542b6bdd...,9,1


In [ ]:
import numpy

In [ ]:
def leave_one_cv(profile_items):
   for item in profile_items:
      item_type = df[df['id'] == item]['type'].tolist()
      if item_type[0] != 'publication':
        users['id'] = users['id'].apply(lambda x:[i for i in x if i != item])
        return item
      

In [ ]:
users['leave_one_cv'] = users['id'].apply(lambda x:leave_one_cv(x))

In [ ]:
users['recommended_items'] = users['id'].apply(lambda s:rec_user1(s)) 

## Evaluation - OLD

In [ ]:
def get_keys(profile):
  mylist = []
  for id_p in profile:
    for id in df['id']:
      if id_p == id:
        mylist.append(df.loc[df['id'] == id, 'keyword'].iloc[0] )
  return mylist

In [ ]:
users['profile_keywords'] = users['profile'].map(lambda x:get_keys(x))

In [ ]:
users['recommended_keywords'] = users['recommended_items'].map(lambda x:get_keys(x))

In [ ]:
users['profile_keywords'] = users['profile_keywords'].map(lambda x:list(set([item if type(sublist) is list else sublist for sublist in x for item in sublist])))

In [ ]:
#users['recommended_keywords'] = users['recommended_keywords'].map(lambda x:list(set([item if type(sublist) is list else sublist for sublist in x for item in sublist])))

In [ ]:
#users['profile_keywords'] = [','.join(map(str, l)) for l in users['profile_keywords']] #list to str


In [ ]:
#users['recommended_keywords'] = [','.join(map(str, l)) for l in users['recommended_keywords']] #list to str


In [ ]:
#users['recommended_keywords'] = users['recommended_keywords'].str.strip('').str.split(',')#str to list


In [ ]:
#users['profile_keywords'] = users['profile_keywords'].str.strip('').str.split(',') #str to list


In [ ]:
def counter(user):
  count = 0
  mylist = []
  for item_list in user['recommended_keywords']:
    for item in item_list:
      if item in user["profile_keywords"]:
        count+=1
    mylist.append(count)
    count = 0
  return mylist
    

In [ ]:
users['counter'] = users.apply(lambda row : counter(row),axis=1)


In [ ]:
pd.set_option('display.max_rows', None)


In [ ]:
def norm_counters(user):
  new_list = []
  for item in user['counter']:
    new_item = item/len(user['profile_keywords'])
    new_list.append(new_item)
  return new_list

In [ ]:
users['norm_counter'] = users.apply(lambda x:norm_counters(x),axis=1)

In [ ]:
my_th = users['norm_counter'][0]

In [ ]:
## Putting threshhold

In [ ]:
def put_thr(user):
  new_counter = []
  thr = 0.1

  for num in user['counter']:
    if num > thr:
      num = 1
      new_counter.append(num)
    else:
      num = 0
      new_counter.append(num)
  return new_counter


In [ ]:
users['relevant_list'] = users.apply(lambda x:put_thr(x),axis=1)

In [ ]:
users.to_json('users_evaluation_3.json')

In [ ]:
users["counter"]

0           [1, 3, 0, 1, 1, 0, 4, 2, 0, 4]
1        [10, 1, 1, 7, 1, 4, 11, 7, 11, 3]
2           [1, 3, 1, 2, 1, 2, 1, 0, 2, 4]
3           [5, 5, 0, 6, 6, 7, 0, 5, 2, 4]
4         [1, 2, 12, 1, 24, 0, 3, 1, 2, 3]
5           [3, 0, 5, 4, 2, 6, 0, 6, 0, 5]
6           [1, 3, 0, 1, 1, 0, 4, 2, 0, 4]
7           [1, 2, 0, 0, 2, 1, 4, 1, 1, 1]
8           [3, 0, 0, 0, 0, 0, 1, 0, 1, 3]
9           [1, 1, 2, 2, 1, 2, 2, 7, 1, 1]
10          [8, 7, 0, 3, 8, 9, 5, 5, 6, 3]
11          [0, 4, 2, 5, 6, 5, 5, 3, 1, 0]
12          [4, 6, 2, 5, 0, 6, 3, 1, 1, 0]
13          [7, 2, 1, 1, 5, 5, 2, 1, 7, 2]
14        [9, 12, 2, 1, 2, 14, 0, 9, 1, 1]
15          [6, 9, 7, 4, 9, 3, 6, 6, 6, 9]
16         [11, 4, 1, 0, 1, 0, 1, 0, 6, 8]
17          [6, 5, 3, 5, 2, 1, 2, 5, 0, 0]
18          [0, 3, 0, 1, 1, 0, 1, 1, 0, 0]
19        [11, 2, 0, 0, 1, 11, 8, 2, 9, 7]
20          [4, 6, 2, 5, 0, 6, 3, 1, 1, 0]
21       [10, 1, 1, 7, 1, 4, 11, 7, 11, 3]
22         [9, 9, 1, 1, 6, 2, 4, 5, 11, 1]
23       [1

## Graph-based Evaluation - OLD

In [ ]:
relations = pd.read_json("/content/gdrive/MyDrive/Recommendation_Systems_Activities/TPDL2022/Data/data_with_relations_v1/relations.json",lines=True)

In [ ]:
users.head()

,user,profile,profile_items,recommended_items,profile_keywords,recommended_keywords,counter
0,"Goeree, Ron",[50|r37980778c78::5d884a4d53183be71aee1935e9de...,[50|r37980778c78::5d884a4d53183be71aee1935e9de...,[50|r37980778c78::1a73ff1dddeee3d70c0f263443c0...,"[temporal, Science Policy, systematic, fmri, e...","[[Neuroscience, Sociology, Developmental Biolo...","[1, 3, 0, 1, 1, 0, 4, 2, 0, 4]"
1,"Ottaviani, Diego",[50|doi_dedup___::b2ff6d882e4c408075d34e755e6c...,[50|doi_dedup___::b2ff6d882e4c408075d34e755e6c...,[50|doi_________::eceb0ae0ffc39497f8c5280cf3dc...,"[Physiology, FOS: Health sciences, Sociology, ...","[[Space Science, Medicine, Cell Biology, Physi...","[10, 1, 1, 7, 1, 4, 11, 7, 11, 3]"
2,"He, Sheng",[50|doi_________::5de5ab6d89e9762cdc7c09f64ae5...,[50|doi_________::5de5ab6d89e9762cdc7c09f64ae5...,[50|r37980778c78::4211c2946bd80183b514184fa60b...,"[Physiology, visual perception, reflects, Medi...","[[Medicine, Biological Sciences, neuroscience,...","[1, 3, 1, 2, 1, 2, 1, 0, 2, 4]"
3,"Kent, James D.",[50|r37980778c78::ff9ef9488ec24e8638aab2e4aeaa...,[50|doi_________::3987780588ec254078aad955fb41...,[50|doi_________::64abcbec05fcc66a18708ab2ae84...,"[Physiology, Sociology, Medicine, Science Poli...","[[neuroimaging, betaseries, analysis, fMRI, BI...","[5, 5, 0, 6, 6, 7, 0, 5, 2, 4]"
4,"Ioanas, Horea-Ioan",[50|doi_dedup___::4566d7cf98d9c145aa76997bc61b...,[50|doi_dedup___::4566d7cf98d9c145aa76997bc61b...,[50|doi_dedup___::5c9f46e8b70db1991c613f4b9bca...,"[Physiology, Sociology, rodents, Medicine, pre...","[[Genetics, FOS: Biological sciences, Develop ...","[1, 2, 12, 1, 24, 0, 3, 1, 2, 3]"


In [ ]:
users['profile_items'][0]

['50|r37980778c78::5d884a4d53183be71aee1935e9deb69d',
 '50|r37980778c78::4e164f63539c6271c0501cd9aa2ffb1c',
 '50|r37980778c78::2db82072751e1d4138e13e552155ad26',
 '50|r37980778c78::6ec9417e65f405165fd98bc7162a2e70',
 '50|r37980778c78::8e1cfb89b795595821b39cb401af9dcb',
 '50|r37980778c78::1a63366370fc8fd83a490e721f7eb2a9',
 '50|r37980778c78::e31b4a4561029b3619d7ed2bcacd42b1',
 '50|r37980778c78::4e1644dc866ecc6f9a13fd750c868238',
 '50|r37980778c78::868677027caa38c2db1ddacce7e53d72',
 '50|r37980778c78::5ba3e79c05baeb92fcd3d7c9eed664e6',
 '50|r37980778c78::db8cd6a8f3cbd3707ce6c78eaaabcaa5',
 '50|r37980778c78::9e36793997275ae1a9ebf52b84a57a90',
 '50|r37980778c78::c3040539c1bbfe6df425c957ce212493']

In [ ]:
def evaluation(user_profile,rec_to_user):
  hits = [] 
  # users_related_items = list(pd.merge(user,relations,left_on='profile',right_on='source')['target'])
  # users_related_items.append(list(pd.merge(user,relations,left_on='profile',right_on='target')['source']))
  relations['indicator'] = relations['source'].isin(user_profile)
  target_items = relations.loc[relations['indicator'] == True]['target']
  relations['indicator'] = relations['target'].isin(user_profile)
  source_items = relations.loc[relations['indicator'] == True]['source']
  for item in rec_to_user:
    contains = False
    if item in list(source_items):
      contains = True
    if item in list(target_items):
      contains = True
    hits.append(contains)
  return hits

In [ ]:
users['graph_evaluation'] = users.apply(lambda row : evaluation(row['profile_items'],row['recommended_items']),axis=1)

ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok


In [ ]:
users['graph_evaluation']

0      [False, False, False, False, False, False, Fal...
1      [False, False, False, False, False, False, Fal...
2      [False, False, False, False, False, False, Fal...
3      [False, False, False, False, False, False, Fal...
4      [False, False, False, False, False, False, Fal...
                             ...                        
113    [True, False, True, False, False, False, False...
114    [False, False, False, False, False, False, Fal...
115    [False, False, False, False, False, False, Fal...
116    [False, False, False, False, False, False, Fal...
117    [False, False, False, False, False, False, Fal...
Name: graph_evaluation, Length: 118, dtype: object

In [ ]:
users.loc[0].to_frame()

,0
user,"Goeree, Ron"
profile,[50|r37980778c78::5d884a4d53183be71aee1935e9de...
profile_items,[50|r37980778c78::5d884a4d53183be71aee1935e9de...
recommended_items,[50|r37980778c78::1a73ff1dddeee3d70c0f263443c0...
profile_keywords,"[temporal, Science Policy, systematic, fmri, e..."
recommended_keywords,"[[Neuroscience, Sociology, Developmental Biolo..."
counter,"[1, 3, 0, 1, 1, 0, 4, 2, 0, 4]"


## Hit_rate OLD

In [ ]:
def hit_rate(row):
  for item in row["recommended_items"]:
    if item == row['leave_one_cv']:
      return 1
  return 0 


In [ ]:
users['hit_rate'] = users.apply(lambda x:hit_rate(x),axis=1)

In [ ]:
users['hit_rate'].mean()

0.30033003300330036